<h1>Load Himawari-8 AHI data and generate true color RGB</h1>

In [1]:
from satpy.scene import Scene
from satpy.resample import get_area_def
from datetime import datetime

<p>We load the data using the Himawari standard format reader</p>

In [2]:
scn = Scene(
      sensor='ahi',
      start_time=datetime(2015, 2, 7, 3, 0),
      end_time=datetime(2015, 2, 7, 3, 10),
      base_dir="/home/a000680/data/himawari8/201502070300",
      reader='ahi_hsd')

<p>Here we do not use the high resolution data</p>

In [3]:
rgbname = 'true_color_reducedsize'

In [4]:
scn.load([rgbname])

In [5]:
scn.show(rgbname)

/home/a000680/usr/src/satpy/satpy/enhancements/__init__.py:51: RuntimeWarning: divide by zero encountered in log10
  np.log10(chn.data, out=chn.data)
/home/a000680/usr/src/satpy/satpy/enhancements/__init__.py:51: RuntimeWarning: invalid value encountered in log10
  np.log10(chn.data, out=chn.data)


<img src='./himawari_ahi_truecolor_201502070300_thumb.png'>

<img href='himawari_ahi_truecolor_201502070300.png'>

In [6]:
scn.save_dataset(rgbname, filename='himawari_ahi_truecolor_{datetime}.png'.format(datetime=scn.start_time.strftime('%Y%m%d%H%M')))

In [7]:
areadef = get_area_def("japan")

In [8]:
local_scene = scn.resample(areadef)

In [9]:
local_scene.show(rgbname)

<img src='./himawari_ahi_truecolor_201502070300_japan_thumb.png'>